In [3]:
# import the necessary packages
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
def midpoint(ptA, ptB):
	return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)
# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
# 	help="path to the input image")
# ap.add_argument("-w", "--width", type=float, required=True,
# 	help="width of the left-most object in the image (in inches)")
# args = vars(ap.parse_args())


In [6]:
#--image images/example_01.png --width 0.955
img="test1.jpg"
image = cv2.imread(img)
width=0.955

In [4]:
# load the image, convert it to grayscale, and blur it slightly
image = cv2.imread(img)
#image = cv2.imread(args["image"])

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)
# perform edge detection, then perform a dilation + erosion to
# close gaps in between object edges
edged = cv2.Canny(gray, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)
# find contours in the edge map
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
# sort the contours from left-to-right and initialize the
# 'pixels per metric' calibration variable
(cnts, _) = contours.sort_contours(cnts)
pixelsPerMetric = None

In [7]:
# loop over the contours individually
for c in cnts:
	# if the contour is not sufficiently large, ignore it
	if cv2.contourArea(c) < 100:
		continue
	# compute the rotated bounding box of the contour
	orig = image.copy()
	box = cv2.minAreaRect(c)
	box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
	box = np.array(box, dtype="int")
	# order the points in the contour such that they appear
	# in top-left, top-right, bottom-right, and bottom-left
	# order, then draw the outline of the rotated bounding
	# box
	box = perspective.order_points(box)
	cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
	# loop over the original points and draw them
	for (x, y) in box:
		cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)
		# unpack the ordered bounding box, then compute the midpoint
	# between the top-left and top-right coordinates, followed by
	# the midpoint between bottom-left and bottom-right coordinates
	(tl, tr, br, bl) = box
	(tltrX, tltrY) = midpoint(tl, tr)
	(blbrX, blbrY) = midpoint(bl, br)
	# compute the midpoint between the top-left and top-right points,
	# followed by the midpoint between the top-righ and bottom-right
	(tlblX, tlblY) = midpoint(tl, bl)
	(trbrX, trbrY) = midpoint(tr, br)
	# draw the midpoints on the image
	cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
	cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)
	# draw lines between the midpoints
	cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
		(255, 0, 255), 2)
	cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
		(255, 0, 255), 2)
# compute the Euclidean distance between the midpoints
	dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
	dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
	# if the pixels per metric has not been initialized, then
	# compute it as the ratio of pixels to supplied metric
	# (in this case, inches)
	if pixelsPerMetric is None:
		pixelsPerMetric = dB / width
	# compute the size of the object
	dimA = dA / pixelsPerMetric
	dimB = dB / pixelsPerMetric
	# draw the object sizes on the image
	cv2.putText(orig, "{:.1f}in".format(dimA),
		(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)
	cv2.putText(orig, "{:.1f}in".format(dimB),
		(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)
	# show the output image
	cv2.imshow("Image", orig)
	cv2.waitKey(0)

KeyboardInterrupt: 

In [9]:
cv2.destroyAllWindows() 


In [ ]:
# compute the Euclidean distance between the midpoints
	dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
	dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))
	# if the pixels per metric has not been initialized, then
	# compute it as the ratio of pixels to supplied metric
	# (in this case, inches)
	if pixelsPerMetric is None:
		pixelsPerMetric = dB / args["width"]
	# compute the size of the object
	dimA = dA / pixelsPerMetric
	dimB = dB / pixelsPerMetric
	# draw the object sizes on the image
	cv2.putText(orig, "{:.1f}in".format(dimA),
		(int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)
	cv2.putText(orig, "{:.1f}in".format(dimB),
		(int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (255, 255, 255), 2)
	# show the output image
	cv2.imshow("Image", orig)
	cv2.waitKey(0)

In [10]:
from ultralytics import YOLO
import cv2
import math 
# start webcam
img="test2.png"
image = cv2.imread(img)
# model
model = YOLO("yolo-Weights/yolov8n.pt")

# object classes
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]


#success, img = cap.read()
results = model(img, stream=True)

# coordinates
for r in results:
    boxes = r.boxes

    for box in boxes:
        # bounding box
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

        # put box in cam
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

        # confidence
        confidence = math.ceil((box.conf[0]*100))/100
        print("Confidence --->",confidence)

        # class name
        cls = int(box.cls[0])
        print("Class name -->", classNames[cls])

        # object details
        org = [x1, y1]
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 1
        color = (255, 0, 0)
        thickness = 2

        cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

cv2.imshow('Webcam', img)
cv2.waitKey(1)
#     break

# cap.release()
# cv2.destroyAllWindows()

KeyboardInterrupt: 

In [11]:
from ultralytics import YOLO
import cv2
import math 
# start webcam
img="test2.png"
image = cv2.imread(img)
# model
model = YOLO("yolo-Weights/yolov8n.pt")

AttributeError: partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

In [38]:
import cv2
def gradient_thresh(img, thresh_min = 25, thresh_max = 100):
    """
    Apply sobel edge detection on input image in x, y direction
    """
    #1. Convert the image to gray scale
    #2. Gaussian blur the image
    #3. Use cv2.Sobel() to find derievatives for both X and Y Axis
    #4. Use cv2.addWeighted() to combine the results
    #5. Convert each pixel to uint8, then apply threshold to get binary image

    ## TODO
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray_blur = cv2.GaussianBlur(img_gray,(11,11),0)

    grad_x = cv2.Sobel(img_gray_blur, cv2.CV_16S, 1, 0, ksize=3)
    grad_y = cv2.Sobel(img_gray_blur, cv2.CV_16S, 0, 1, ksize=3)


    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)


    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    #cv2.imshow("grad",grad)
    max_val=1
    _,binary_output1 = cv2.threshold(grad,90,max_val,cv2.THRESH_BINARY)
    _,binary_output2 = cv2.threshold(grad,130,max_val,cv2.THRESH_BINARY)
    binary_output = binary_output1 - binary_output2
    cv2.imshow("binary_output",binary_output)
    cv2.waitKey(0)
    return binary_output

In [32]:
image=cv2.imread("illinois.jpg")

In [33]:
filter_im=gradient_thresh(image)


In [39]:
from skimage import morphology

binaryImage = morphology.remove_small_objects(filter_im.astype('bool'),min_size=50,connectivity=20)


In [40]:
filter_im=binaryImage

In [43]:
import numpy as np
filter_im=np.array(filter_im)
indices = np.argwhere(filter_im == 1)

In [44]:
print(len(indices))

15403


In [45]:
for i in indices:
    center = (i[1],i[0])
    radius = 1
    color = (0, 0, 255)  # Red color in BGR format

    # Draw the dot on the image
    cv2.circle(image, center, radius, color, -1)  # -1 fills the circle with color

    # Display the image
    cv2.imshow('Image with Dot', image)
    cv2.waitKey(5)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [36]:
print(len(indices))

31923


In [13]:
mult=2

In [42]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect lines using Hough Line Transform
lines = cv2.HoughLinesP(gray, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

# Extract coordinates of lines
line_coords = []
for line in lines:
    x1, y1, x2, y2 = line[0]
    line_coords.append([(x1, y1), (x2, y2)])

# Fit a polygon to the lines
polygon = cv2.fitLine(np.array(line_coords), cv2.DIST_L2, 0, 0.01, 0.01)

# Draw the polygon
(x0, y0) = polygon[2], polygon[3]
(dx, dy) = polygon[0], polygon[1]
cv2.line(image, (int(x0 - 1000*dx), int(y0 - 1000*dy)), (int(x0 + 1000*dx), int(y0 + 1000*dy)), (0, 255, 0), 2)

# Display the result
cv2.imshow('Result', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\linefit.cpp:618: error: (-215:Assertion failed) npoints2 >= 0 || npoints3 >= 0 in function 'cv::fitLine'


: 

In [14]:
import numpy as np

# Example 2D array
array_2d = np.array([[0, 1, 0],
                     [1, 0, 1],
                     [1, 1, 0]])

# Get indices where value is 1
indices = np.argwhere(array_2d == 1)

print("Indices where value is 1:")
print(indices)


Indices where value is 1:
[[0 1]
 [1 0]
 [1 2]
 [2 0]
 [2 1]]


#start

In [175]:
import cv2

image = cv2.imread("IMG_0021.jpg")
new_width = 500
new_height = 300

# Resize the image
image = cv2.resize(image, (new_width, new_height))

filtered_image = cv2.GaussianBlur(image, (5, 5), 0)
edges = cv2.Canny(filtered_image, 100, 200)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [153]:
a4_width=0.297
im_width=image.shape[1]
scale=a4_width/im_width

In [154]:
print(scale)

0.0002475


In [148]:
print(image.shape[1])

1200


In [ ]:
Or = 240
Oc = 320
alpha = 0.013349570451068913
beta = 732.268
world=[270.38157653808594, 51.403507232666016]
#world=[262.2084503173828, 51.064043045043945]
# Function that converts image coord to world coord
Or=world[1]
Oc=world[0]
t_x=0.25
t_y=0.08


def IMG2W(col, row):
    x_c=(row-Or)/beta
    y_c=(col-Oc)/beta
    #print('x_c   ',x_c)
    #print('y_c   ',y_c)

    x_w=x_c*np.cos(alpha)+y_c*np.sin(alpha)
    y_w=-x_c*np.sin(alpha)+y_c*np.cos(alpha)
    return [x_w,y_w]

In [ ]:
move_arm(pub_cmd, loop_rate, dest, vel, accel)

In [ ]:
two_d_array=

In [158]:
#convert 
original_array=np.array(contours)
two_d_array = original_array.reshape(original_array.shape[0], -1)
print(original_array)
reshaped_array = original_array.reshape(original_array.shape[0], original_array.shape[1], -1)

print(reshaped_array)

C:\Users\nrodw\AppData\Local\Temp\ipykernel_13068\266148831.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  original_array=np.array(contours)


IndexError: tuple index out of range

In [176]:

#image = cv2.imread("IMG_0021.jpg")

#i=8
for i in range(len(contours)):
# radius = 3  # Adjust the radius to control the size of the dot
# color = (255, 0, 0)  # Green in BGR
    
# # Draw the dot on the image
#     #image = cv2.circle(image, center_coordinates, radius, color, -1)
#     #cv2.line(image, start_point, end_point, color,5 )
#     # if j==len(contours[i])-1:
#     #     end_point=(contours[i][0][0][0],contours[i][0][0][1])
#     # else:
#     #     end_point=(contours[i][j+1][0][0],contours[i][j+1][0][1])
# start_point=(contours[i][0][0][0],contours[i][0][0][1])
# cv2.circle(image, start_point, radius, color, -1)
# start_point=(contours[i][len(contours[i])-1][0][0],contours[i][len(contours[i])-1][0][1])
# cv2.circle(image, start_point, radius, color, -1)
    for j in range(len(contours[i])):
        start_point=(contours[i][j][0][0],contours[i][j][0][1])
        
        print(start_point)
        radius = 3  # Adjust the radius to control the size of the dot
        color = (0, 255, 0)  # Green in BGR

    # Draw the dot on the image
        #image = cv2.circle(image, center_coordinates, radius, color, -1)
        #cv2.line(image, start_point, end_point, color,5 )
        if j==len(contours[i])-1:
            end_point=(contours[i][0][0][0],contours[i][0][0][1])
        else:
            end_point=(contours[i][j+1][0][0],contours[i][j+1][0][1])

        cv2.line(image, start_point, end_point, color,5)
cv2.imshow('Contours', image) 
cv2.waitKey(0)
# cv2.imshow('Contours', image) 
# cv2.waitKey(0) 

(438, 292)
(436, 294)
(435, 294)
(435, 296)
(434, 297)
(427, 297)
(427, 298)
(428, 297)
(435, 297)
(435, 295)
(437, 293)
(438, 293)
(439, 292)
(440, 292)
(7, 287)
(6, 288)
(4, 288)
(3, 289)
(2, 289)
(1, 290)
(0, 290)
(1, 290)
(2, 289)
(4, 289)
(5, 288)
(7, 288)
(8, 287)
(9, 287)
(321, 286)
(321, 290)
(320, 291)
(320, 294)
(322, 296)
(323, 296)
(323, 295)
(322, 295)
(320, 293)
(320, 291)
(321, 290)
(321, 287)
(322, 286)
(464, 280)
(463, 281)
(463, 285)
(460, 288)
(456, 288)
(451, 293)
(445, 293)
(443, 291)
(445, 289)
(444, 289)
(443, 290)
(443, 291)
(442, 292)
(442, 294)
(441, 295)
(440, 295)
(439, 296)
(438, 296)
(438, 297)
(440, 295)
(442, 295)
(442, 294)
(444, 292)
(446, 294)
(446, 296)
(446, 294)
(447, 293)
(451, 293)
(453, 291)
(454, 291)
(457, 288)
(458, 288)
(459, 289)
(460, 288)
(461, 288)
(463, 286)
(463, 282)
(465, 280)
(467, 280)
(472, 273)
(471, 274)
(469, 274)
(468, 275)
(466, 275)
(465, 276)
(464, 276)
(463, 277)
(462, 277)
(460, 279)
(459, 279)
(458, 280)
(457, 280)
(456,

32

In [171]:
image = cv2.imread("illinois.jpg")

#i=8
#match to the work space assume know 4 corners

#1 pixel = 0.0005 m (half cm)
tl=np.array([0.1,0.2])
dest = (0.00913275, 0.0746955, 0)

for i in range(len(contours)):
    #top of first point
    two_d_array = contours[i].reshape(contours[i].shape[0], -1)
    two_d_array=two_d_array*scale
    number_to_subtract_column1 = two_d_array[0][0]
    number_to_subtract_column2 = two_d_array[0][1]
    print('two_d_array  ',two_d_array)

    # Subtract the numbers from each column
    result_array = (two_d_array - np.array(two_d_array[0]))
    result_array=result_array+tl
    #start_xw_yw_zw=[tl+]
    #start_loc=lab_invk(start_xw_yw_zw[0], start_xw_yw_zw[1], start_xw_yw_zw[2],0)
    dest_start=(result_array[0][0],result_array[0][1],0.01)    
    #move_arm(pub_cmd, loop_rate, dest_start, vel, accel)
    print('result_array  ',result_array)
    
    for j in range(len(result_array)):
        #print(result_array[j][0],result_array[j][1])
        dest=(result_array[j][0],result_array[j][1],0)
        print('dest ',dest)
        #print(start_point)
        radius = 3  # Adjust the radius to control the size of the dot
        color = (0, 255, 0)  # Green in BGR
        #move_arm(pub_cmd, loop_rate, dest, vel, accel)
    
    
    #move to first point again
    dest=(result_array[0][0],result_array[0][1],0)
    #move_arm(pub_cmd, loop_rate, dest, vel, accel)
    #move up
    #move_arm(pub_cmd, loop_rate, dest_start, vel, accel)

#move_arm(pub_command, loop_rate, go_away, vel, accel)


two_d_array   [[0.248985  0.08118  ]
 [0.2487375 0.0814275]
 [0.2482425 0.0814275]
 [0.2482425 0.116325 ]
 [0.24849   0.1165725]
 [0.255915  0.1165725]
 [0.2561625 0.116325 ]
 [0.2561625 0.0814275]
 [0.2556675 0.0814275]
 [0.25542   0.08118  ]]
result_array   [[0.1       0.2      ]
 [0.0997525 0.2002475]
 [0.0992575 0.2002475]
 [0.0992575 0.235145 ]
 [0.099505  0.2353925]
 [0.10693   0.2353925]
 [0.1071775 0.235145 ]
 [0.1071775 0.2002475]
 [0.1066825 0.2002475]
 [0.106435  0.2      ]]
dest  (0.1, 0.2, 0)
dest  (0.0997525, 0.20024750000000002, 0)
dest  (0.0992575, 0.20024750000000002, 0)
dest  (0.0992575, 0.235145, 0)
dest  (0.09950499999999998, 0.2353925, 0)
dest  (0.10693, 0.2353925, 0)
dest  (0.10717750000000001, 0.235145, 0)
dest  (0.10717750000000001, 0.20024750000000002, 0)
dest  (0.10668249999999999, 0.20024750000000002, 0)
dest  (0.10643499999999997, 0.2, 0)
two_d_array   [[0.16533   0.08118  ]
 [0.1650825 0.0814275]
 [0.1645875 0.0814275]
 [0.1645875 0.116325 ]
 [0.164835  0.1

In [159]:
import numpy as np

# Example 2D array
array_2d = np.array([[1, 2],
                     [3, 4],
                     [5, 6]])

# Numbers to subtract from the first and second columns
number_to_subtract_column1 = 10
number_to_subtract_column2 = 20

# Subtract the numbers from each column
result_array = array_2d - np.array([number_to_subtract_column1, number_to_subtract_column2])

print("Original array:")
print(array_2d)
print("\nResulting array:")
print(result_array)


Original array:
[[1 2]
 [3 4]
 [5 6]]

Resulting array:
[[ -9 -18]
 [ -7 -16]
 [ -5 -14]]


In [72]:
import numpy as np

In [125]:
image = cv2.imread("illinois.jpg")

i=8
original_array=np.array(contours[i])
two_d_array = original_array.reshape(original_array.shape[0], -1)
print(len(two_d_array))
outliers = detect_outliers_with_kmeans(two_d_array, 100)
print(len(outliers))

for j in range(len(contours[i])):
    start_point=(contours[i][j][0][0],contours[i][j][0][1]) 
    #print(start_point)
    radius = 3  # Adjust the radius to control the size of the dot
    color = (0, 255, 0)  # Green in BGR
    
# Draw the dot on the image
    #image = cv2.circle(image, center_coordinates, radius, color, -1)
    #cv2.line(image, start_point, end_point, color,5 )
    # if j==len(contours[i])-1:
    #     end_point=(contours[i][0][0][0],contours[i][0][0][1])
    # else:
    #     end_point=(contours[i][j+1][0][0],contours[i][j+1][0][1])
    cv2.circle(image, start_point, radius, color, -1)

for j in range(len(outliers)):
    print(outliers[j])
    start_point=(outliers[j][0],outliers[j][1]) 
    print(start_point)
    radius = 3  # Adjust the radius to control the size of the dot
    color = (0, 0, 255)  # Green in BGR
    
# Draw the dot on the image
    #image = cv2.circle(image, center_coordinates, radius, color, -1)
    #cv2.line(image, start_point, end_point, color,5 )
    # if j==len(contours[i])-1:
    #     end_point=(contours[i][0][0][0],contours[i][0][0][1])
    # else:
    #     end_point=(contours[i][j+1][0][0],contours[i][j+1][0][1])
    #cv2.circle(image, start_point, radius, color, -1)
    #cv2.line(image, start_point, end_point, color,5)
cv2.imshow('Contours', image) 
cv2.waitKey(0)


28
6
[ 37 302]
(37, 302)
[ 36 303]
(36, 303)
[ 36 496]
(36, 496)
[ 37 497]
(37, 497)
[171 497]
(171, 497)
[171 302]
(171, 302)


-1

In [95]:
outliers = detect_outliers_with_kmeans(two_d_array, 80)
print(len(outliers))
start_point=(contours[i][j][0][0],contours[i][j][0][1])
    
    print(start_point)
    radius = 3  # Adjust the radius to control the size of the dot
    color = (0, 255, 0)  # Green in BGR

# Draw the dot on the image
    #image = cv2.circle(image, center_coordinates, radius, color, -1)
    cv2.line(image, start_point, end_point, color,5 )
    if j==len(contours[i])-1:
        end_point=(contours[i][0][0][0],contours[i][0][0][1])
    else:
        end_point=(contours[i][j+1][0][0],contours[i][j+1][0][1])
    cv2.line(image, start_point, end_point, color,5)
cv2.imshow('Contours', image) 
cv2.waitKey(0)


IndentationError: unexpected indent (3514549919.py, line 5)

In [11]:
print(contours[0])

[[[1006  328]]

 [[1005  329]]

 [[1003  329]]

 [[1003  470]]

 [[1004  471]]

 [[1034  471]]

 [[1035  470]]

 [[1035  329]]

 [[1033  329]]

 [[1032  328]]]


print(contours)

In [81]:
from sklearn.cluster import KMeans
import numpy as np

def detect_outliers_with_kmeans(points, threshold):
    # Fit KMeans with one cluster
    kmeans = KMeans(n_clusters=1).fit(points)
    
    # Get the centroid
    centroid = kmeans.cluster_centers_[0]
    
    # Calculate distances of each point from the centroid
    distances = np.linalg.norm(points - centroid, axis=1)
    
    # Identify outliers based on the threshold
    outliers = [point for point, distance in zip(points, distances) if distance > threshold]
    
    return outliers

# Example usage:
# points is your array of 2D points
# threshold is the distance threshold beyond which points are considered outliers
threshold=10


In [14]:
cv2.imshow('Canny Edges After Contouring', edges) 
cv2.waitKey(0) 
  
print("Number of Contours found = " + str(len(contours))) 
  
# Draw all contours 
# -1 signifies drawing all contours 
cv2.drawContours(image, contours, -1, (0, 255, 0), 3) 
  
cv2.imshow('Contours', image) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

Number of Contours found = 9


In [65]:
cv2.destroyAllWindows() 

In [25]:
cv2.imshow('Contours', image_with_dot) 
cv2.waitKey(0) 
#cv2.destroyAllWindows() 

-1

In [177]:
cv2.destroyAllWindows() 